In [53]:
import autograd.numpy as np
from autograd.core import primitive
from autograd import grad, jacobian, hessian
from autograd.numpy.numpy_grads import unbroadcast
import scipy.stats

In [35]:
@primitive
def MySum(x):
    return 2 * np.sum(x)

def MySameSum(x):
    return 2 * np.sum(x)

def MySum_vjp(g, ans, vs, gvs, x):
    return np.full(x.shape, g) * np.full(x.shape, 2)

MySum.defvjp(MySum_vjp)


In [112]:
@primitive
def BinSum(x, y):
    return y * x ** 2

# It appears that the gradient is always with respect to the argument specified in argnum,
# which defaults to zero (the first argument)

def BinSum_vjp_x(g, ans, vs, gvs, x, y):
    return unbroadcast(vs, gvs, g * 2 * x * y)

def BinSum_vjp_y(g, ans, vs, gvs, x, y):
    return unbroadcast(vs, gvs, g * 2 * x ** 2)

BinSum.defvjp(BinSum_vjp_x, argnum=0)
BinSum.defvjp(BinSum_vjp_y, argnum=1)

BinSumGradX = grad(BinSum)
BinSumGradY = grad(BinSum, argnum=1)
print BinSum.vjps
print BinSum(5., 0.1)
print BinSumGradX(5., 0.1)
print BinSumGradY(5., 0.1)


{0: <function VJP_0_of_BinSum at 0x7f966fcc8c08>, 1: <function VJP_1_of_BinSum at 0x7f966fcf09b0>}
2.5
1.0
50.0


In [38]:
x = np.array([5, 1])
y = np.expand_dims(x, 5)
print x.shape
print y.shape
print x ** 2

(2,)
(2, 1)
[25  1]


In [89]:
MySumGrad = grad(MySum)
MySameSumGrad = grad(MySameSum)

x = np.array([2., 4.])
print MySumGrad(x)
print MySameSumGrad(x)

x_full = np.full(13, x.shape)
print x_full
print x_full.shape

print MySum.vjps[0]

[ 2.  2.]
[ 2.  2.]
[2 2 2 2 2 2 2 2 2 2 2 2 2]
(13,)
<function VJP_0_of_MySum at 0x7f966fcf07d0>


In [54]:
@primitive
def SumSq(x):
    return np.sum(x ** 2)

def SumSq_vjp(g, ans, vs, gvs, x):
    print 'g: '
    print g
    print 'ans: '
    print ans
    print 'vs: '
    print vs
    print 'gvs: '
    print gvs
    print 'x: '
    print x
    print 'Returning.'
    return np.full(x.shape, g) * 2 * x

SumSq.defvjp(SumSq_vjp)

def MyFun(x):
    return 3 * SumSq(x)

SumSqGrad = grad(SumSq)
MyFunGrad = grad(MyFun)
MyFunHess = hessian(MyFun)

x = np.array([2., 4.])
print 'Stand alone:'
print SumSqGrad(x)

print '\nIn function:'
print MyFunGrad(x)

print '\nIn Hessian:'
print MyFunHess(x)

Stand alone:
g: 
1.0
ans: 
Autograd ArrayNode with value 20.0 and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': <type 'float'>, 'size': 1}
x: 
Autograd ArrayNode with value [ 2.  4.] and 1 progenitors(s)
Returning.
[ 4.  8.]

In function:
g: 
3.0
ans: 
Autograd ArrayNode with value 20.0 and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': <type 'float'>, 'size': 1}
x: 
Autograd ArrayNode with value [ 2.  4.] and 1 progenitors(s)
Returning.
[ 12.  24.]

In Hessian:
g: 
3.0
ans: 
Autograd ArrayNode with value 20.0 and 2 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': 

In [51]:
from autograd import jacobian

def MyVecFun(x):

    return np.array([ SumSq(x), 5 * SumSq(x) ])

MyVecJac = jacobian(MyVecFun)
MyVecJac(x)

g: 
0.0
ans: 
Autograd ArrayNode with value 20.0 and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': <type 'float'>, 'size': 1}
x: 
Autograd ArrayNode with value [ 2.  4.] and 1 progenitors(s)
Returning.
g: 
1.0
ans: 
Autograd ArrayNode with value 20.0 and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': <type 'float'>, 'size': 1}
x: 
Autograd ArrayNode with value [ 2.  4.] and 1 progenitors(s)
Returning.
g: 
5.0
ans: 
Autograd ArrayNode with value 20.0 and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (), 'scalartype': <type 'float'>, 'size': 1}
x: 
Autograd ArrayNode with value [

array([[  4.,   8.],
       [ 20.,  40.]])

In [71]:
@primitive
def SqVec(x):
    return np.hstack([ x ** 2, 2 * x ])

def SqVec_vjp(g, ans, vs, gvs, x):
    print 'g: '
    print g
    print 'ans: '
    print ans
    print 'vs: '
    print vs
    print 'gvs: '
    print gvs
    print 'x: '
    print x
    print 'Returning.'
    return np.matmul(g * np.hstack([ 2 * x, np.full(2, x.shape) ]))

SqVec.defvjp(SqVec_vjp)

SqVecJac = jacobian(SqVec)

x = np.array([3., 4.])
print SqVec(x)
print SqVecJac(x)


[  9.  16.   6.   8.]
g: 
[ 1.  0.  0.  0.]
ans: 
Autograd ArrayNode with value [  9.  16.   6.   8.] and 1 progenitors(s)
vs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (2,), 'scalartype': <type 'float'>, 'size': 2}
gvs: 
ArrayVSpace_{'dtype': dtype('float64'), 'shape': (4,), 'scalartype': <type 'float'>, 'size': 4}
x: 
Autograd ArrayNode with value [ 3.  4.] and 1 progenitors(s)
Returning.


TypeError: Required argument 'b' (pos 2) not found

In [87]:
import numpy as onp
print dir(onp.add)
print dir(onp.add.__call__)
print onp.add(1, 2)

['__call__', '__class__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__name__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'accumulate', 'at', 'identity', 'nargs', 'nin', 'nout', 'ntypes', 'outer', 'reduce', 'reduceat', 'signature', 'types']
['__call__', '__class__', '__cmp__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__name__', '__new__', '__objclass__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
3
